In [ ]:
from common_all import *
from common_float import *

In [ ]:
def FUNC2(x, y): return func(y, x) / K  # 45 9

Simpson's integration:
$
I^* = \frac{h}{6} \sum_i \left(f_i + 4f_{i+0.5} + f_{i+1}\right)
$

In [ ]:
INTEGR_STEP = 1e-2

_INTEGR_STEP_2 = INTEGR_STEP / 2.0

def integrate_segm(x1: int, y1: int, x2: int, y2: int):
    dx, dy = abs(x2 - x1), abs(y2 - y1)
    segm_len = m.hypot(dx, dy)
    halfx = _INTEGR_STEP_2 * dx
    halfy = _INTEGR_STEP_2 * dy

    s = 0
    ptx, pty = x1, y1
    lastval = FUNC2(ptx, pty)

    for i in range(int(segm_len / INTEGR_STEP)):
        # s += FUNC2(ptx, pty)
        s += lastval

        s += 4 * FUNC2(ptx + halfx, pty + halfy)

        ptx += INTEGR_STEP
        pty += INTEGR_STEP
        
        # s += FUNC2(ptx, pty)
        lastval = FUNC2(ptx, pty)
        s += lastval

    res = s * INTEGR_STEP / 6.0
    return res

In [ ]:
# Monte-carlo method (floating)

def prob_trace(y: float, x: float, eps: float, fixed_step):
    _2_PI = 2 * m.pi

    hops = 0
    res = 0

    dist, btemp = dist_to_border(y, x)
    while dist > eps and not is_outside(y, x):
        angle = _2_PI * rnd.random()
        if fixed_step is None:
            nx = x + dist * m.cos(angle)
            ny = y + dist * m.sin(angle)
        else:
            nx = x + fixed_step * m.cos(angle)
            ny = y + fixed_step * m.sin(angle)

        res += integrate_segm(x, y, nx, ny) * 0.25
        x, y = nx, ny

        dist, btemp = dist_to_border(y, x)
        hops += 1
    
    res += btemp
    return res

def prob_point(j, i, traces_cnt: int, eps: float, fixed_step):
    res = 0
    for _ in range(traces_cnt):
        res += prob_trace(j, i, eps, fixed_step)
    res /= traces_cnt
    return res

In [ ]:
TARGET = (45, 9)
TRACES_COUNT = 100
EPS = 1e-2

# Set fixed step for using semi-floating walk
# FIXED_STEP = None
FIXED_STEP = 1

res = prob_point(*TARGET, TRACES_COUNT, EPS, FIXED_STEP)
print('Res: {:.3f}'.format(res))
orig = 330.6864168849065
print('Err: {:.3f}%'.format((abs(res - orig) / orig) * 100))